In [1]:
import re
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

train_df  = pd.read_csv("/kaggle/input/hell-week/train.csv")
test_df = pd.read_csv("/kaggle/input/hell-week/test.csv")

def cleaner(row):
    if row in [np.nan,'nan',0.0]:
        return row
    else:
        x,y=6,-6
        try:
            return float(row[x:y])
        except:
            print(f'{row, type(row)}, exception')

def pre_process(df, mode):
    df = df.drop(['UID', 'ph_no', 'cvv', 'credit_card_number', 'job', 'email', 'url', 'country', 'emoji', 'name'], axis=1)
    n = len(df.columns) if mode == 'test' else -1
    for i in df.columns[:n]:
        df[i] = df[i].apply(cleaner)
    df = df.fillna(df.median())
    return df

train_df=pre_process(train_df,'train')
train_df=train_df.fillna(train_df.median())

categorical_columns = train_df.select_dtypes(include=['object']).columns
train_df.select_dtypes(include=['object']).columns

X=train_df.drop('state',axis=1)
y=train_df['state']



/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = train_df['state']
y_encoded = label_encoder.fit_transform(y)
y_encoded_onehot = np.eye(10)[y_encoded]
print(y_encoded_onehot)



X_train, X_val, y_train, y_val = train_test_split(X, y_encoded_onehot, test_size=0.3)
num_unique_categories = y_encoded_onehot.shape[1]

df_test = pd.read_csv('/kaggle/input/hell-week/test.csv')
df_test_cleaned = pre_process(df_test, 'test')

[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [3]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_unique_categories, activation='softmax')])

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))
predictions = model.predict(df_test_cleaned)
y_pred_labels = np.argmax(predictions, axis=1)
y_pred_decoded = label_encoder.inverse_transform(y_pred_labels)
output = pd.DataFrame({'UID': test_df.UID, 'state':y_pred_decoded})
output.to_csv('submission.csv', index=False)

Epoch 1/20
3063/3063 [==============================] - 8s 2ms/step - loss: 1.3138 - accuracy: 0.5896 - val_loss: 0.8123 - val_accuracy: 0.7290
Epoch 2/20
3063/3063 [==============================] - 7s 2ms/step - loss: 0.6886 - accuracy: 0.7731 - val_loss: 0.6187 - val_accuracy: 0.7970
Epoch 3/20
3063/3063 [==============================] - 7s 2ms/step - loss: 0.5557 - accuracy: 0.8208 - val_loss: 0.5555 - val_accuracy: 0.8228
Epoch 4/20
3063/3063 [==============================] - 7s 2ms/step - loss: 0.4922 - accuracy: 0.8453 - val_loss: 0.5235 - val_accuracy: 0.8342
Epoch 5/20
3063/3063 [==============================] - 7s 2ms/step - loss: 0.4584 - accuracy: 0.8574 - val_loss: 0.5072 - val_accuracy: 0.8408
Epoch 6/20
3063/3063 [==============================] - 7s 2ms/step - loss: 0.4325 - accuracy: 0.8659 - val_loss: 0.5031 - val_accuracy: 0.8440
Epoch 7/20
3063/3063 [==============================] - 8s 3ms/step - loss: 0.4133 - accuracy: 0.8731 - val_loss: 0.4601 - val_accuracy: